In [5]:
! pip install progressbar2

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import re
from progressbar import progressbar
import time
import random

### 상세 검색 페이지에서 파라미터 확인

http://dart.fss.or.kr/dsab002/main.do#  페이지 접속


'주식분할' 키워드로 검색


'검사' 메뉴에서 Form Data 확인


당일 날짜 설정

In [ ]:
# 금일 날짜 설정

dt = datetime.utcnow() + timedelta(hours = 9)    # utc 시간에서 9시간 더함
today = dt.strftime('%Y%m%d')                  # str 형태로 변환
today

In [ ]:
url = 'http://dart.fss.or.kr/dsab002/search.ax'

In [ ]:
data = {
    'currentPage': '1',            # 페이지 1로 충분함
    'maxResults': '100',           # 15에서 100으로 변경
    'maxLinks': '10',
    'sort': 'date',
    'series': 'desc',
    'textCrpCik': '',
    'textCrpNm': '',
    'textPresenterNm': '',
    'startDate': '20160101',
    'endDate': today,         # 오늘 날짜로 변경
    'finalReport': 'recent',
    'typesOfBusiness': 'all',
    'corporationType': 'all',
    'closingAccountsMonth': 'all',
    'reportName': '주식분할'
    }

# headers = {
#     'Origin': 'http://dart.fss.or.kr',
#     'Accept-Encoding': 'gzip, deflate',
#     'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',
#     'Content-Type': 'application/x-www-form-urlencoded',
#     'Accept': 'text/html, */*; q=0.01',
#     'Referer': 'http://dart.fss.or.kr/dsab002/main.do',
#     'X-Requested-With': 'XMLHttpRequest',
#     'Connection': 'keep-alive',
# }


#### 원하는 페이지 요청

In [ ]:
response = requests.post(url, data = data)

In [ ]:
response.text

#### 페이지 응답 여부 확인해보기

In [ ]:
dff = pd.read_html(response.text)[0]

In [ ]:
dff

In [ ]:
dff.shape

In [ ]:
soup = bs(response.text, 'html.parser')

In [ ]:
soup

#### table 코드 확인과 숫자확인

In [ ]:
soup.findAll('table')

In [ ]:
len(soup.findAll('table'))

#### 'tr' tag 첫번째, 마지막번째 요소 확인해보기

In [ ]:
soup.findAll('tr')[2]

In [ ]:
soup.findAll('tr')[1]

In [ ]:
soup.findAll('tr')[-1]

#### tr tag의 숫자 확인하여 실제 공시와 비교해보기

In [ ]:
len(soup.findAll('tr'))

## 첫번째 tr tag의 항목 추출하기

<작업 순서>

* 공시 리스트에서 공시 페이지를 여는 방법 찾아보기
  (iframe 구조 이해 필요)


* 해당 페이지에서 추출하려는 항목을 정리해야 함


* '기재정정' 공시의 구조도 같이 체크할 수 있도록 확인 필요


* 다양한 형태의 자료에 대해서도 추출 가능한 함수 제작 염두


---------------------------------------------------------

<추출 항목 정리>

 - 회사명(company)
 
 
 - 공시일자(announce_date)


 - 분할 전 액면가(far_before)


 - 분할 후 액면가(far_after)


 - 주주총회 예정일(approve_date)


 - 거래정지 첫날(close_date)


 - 신주상장일(open_date)


 - 이사회 결의일(decide_date) - 공시 시간 확인 가능

In [ ]:
# tr[0]의 경우 테이블 라벨이기 때문에 1번부터 진행

work1 = soup.findAll('tr')[2].findAll('td')   

In [ ]:
work1 #td_list

#### 회사명, 공시 일자 체크 

In [ ]:
work1[4].text

In [ ]:
company = work1[1].text.replace('\t','').replace('\n','').replace('\xa0','').replace('\r','')
announce_date = work1[4].text

company, announce_date


#### 종목코드 추출

In [ ]:
code_res = requests.get('http://dart.fss.or.kr' + work1[1].a['href'])

In [ ]:
code = pd.read_html(code_res.text)[0].iloc[3,1]

#### 공시 타이틀 체크('[첨부정정]'으로 시작하는 것은 지우기 위함)

In [ ]:
title = work1[2].text.replace("\n", "").replace("\t","").replace(" ","").replace("\r","")
title

#### 시장 구분 체크('일반법인 제외 목적')

In [ ]:
market = work1[1].img['title']
market

#### 하위 사이트 탐색

In [ ]:
sub_url = work1[2].a['href']
sub_url_1 = 'http://dart.fss.or.kr' + sub_url
sub_url_1

### 하위 사이트 열어보기

* sub_url_1 주소 열어서 내용 확인


* main.do 컨텐츠 확인


* viewer.do 컨텐츠 확인


* viewer.do에 원하는 컨텐츠가 있음 


* 하지만 'dcmNo' 번호에 대한 정보 필요


* 'dcmNo' 의 경우 main.do 에서 찾아보자

#### main.do 사이트 요청 및 컨텐츠 확인(html 구조가 아님)

In [ ]:
sub_main_response = requests.get(sub_url_1)

In [ ]:
sub_main_text = sub_main_response.text

sub_main_text

#### dcmNo 위치를 확인해보자.(ctrl + F 명령어로 검색)

#### dcmNo 위치와 관련하여 반복되는 구문을 찾아보자(201804.......)

#### 2018042.....넘버를 먼저 구한 후 이를 중심으로 split하여 두번째 항목을 살펴보자.

Hint) sub_url의 마지막 부분에 있음

In [ ]:
check_num = sub_url[-14:]
check_num

In [ ]:
sub_main_text.split(check_num)[1]

#### 원하는 번호가 있는 위치를 찾아보자.(노가다로)

In [ ]:
sub_main_text.split(check_num)[1][4:12]

#### 원하는 번호가 있는 위치를 찾아보자.(정규표현식으로)

* 7자리 숫자를 찾아주는 정규표현식은?


* 'python regrex exact n digit number' 로 구글에서 검색


* 정규표현식 문법은 네이버에서 '정규표현식'으로 검색해볼 것

In [ ]:
re.findall(r"\b\d{7}\b",sub_main_text.split(check_num)[1])[0]

In [ ]:
dcmNo = re.findall(r"\b\d{7}\b",sub_main_text.split(check_num)[1])[0]
dcmNo

### dcmNo를 활용하여 viewer.do 문서를 열어보자.

In [ ]:
contents_url = "http://dart.fss.or.kr/report/viewer.do"

#### requests.get의 parameter check

In [ ]:
param_data = {
    'rcpNo': check_num,    # check_num 사용(앞에서 추출한 고유번호)
    'dcmNo': dcmNo,        # dcmNo 사용(앞에서 추출한 고유번호)
    'eleId': '0',
    'offset': '0',
    'length': '0',
    'dtd': 'HTML'
}

#### param을 활용하여 requests 하기

In [2]:
contents_request = requests.get(contents_url, params = param_data)

NameError: name 'contents_url' is not defined

In [3]:
contents_request.text

NameError: name 'contents_request' is not defined

#### 화면 구조 분석

In [6]:
contents_df = pd.read_html(contents_request.text)


NameError: name 'contents_request' is not defined

In [106]:
contents_df[-1]

,0,1,2,3
0,1. 주식분할 내용,구분,분할전,분할후
1,1주당 가액 (원),500,100,NaN
2,발행주식총수,보통주식(주),4553965,22769825
3,종류주식(주),-,-,NaN
4,2. 주식분할 일정,주주총회예정일,2018-12-27,NaN
5,구주권제출기간,시작일,2018-12-28,NaN
6,종료일,2019-01-29,NaN,NaN
7,매매거래정지예정기간,2019-01-28 ~ 2019-02-14,NaN,NaN
8,명의개서정지기간,2019-01-30 ~ 2019-02-13,NaN,NaN
9,신주권교부예정일,2019-02-14,NaN,NaN


In [107]:
len(contents_df)

1

----------


### <추출 항목 정리>

 - 회사명(company)
 
 
 - 공시일자(announce_date)


 - 분할 전 액면가(far_before)


 - 분할 후 액면가(far_after)


 - 주주총회 예정일(approve_date)


 - 거래정지 첫날(close_date)


 - 신주상장일(open_date)


 - 이사회 결의일(decide_date) - 공시 시간 확인 가능

------

### (매우 중요) 주요 명심사항!

* 이번 document 뿐만 아니라 다른 공시에 대해서도 적용이 되어야 함!


* 경우의 수를 잘 생각해 보자.


* 지금 최대한 만들어도 나중에 바꿔야할 가능성 99.999% 이니 일단 한번 해보고 수정하자.

---



1) 정정 공시일 경우 관련 table 이 추가됨


2) 매매거래정지기간의 경우 날짜 양식이 다를 경우


3) 정정 신고 내용에 주요 항목이 표시된 경우


4) 항목이 다른 경우

#### (매우 중요) table 항목을 찾아 가장 마지막 item을 선택하여 작업하도록 함

* 정정 공시의 경우 정정 이전 또는 정정 관련 사항이 가장 위에 위치함


* 마지막 table을 선택하여 작업할 경우 정정 이후 공시만 추출할 수 있음

In [41]:
contents_df[-1]

,0,1,2,3
0,1. 주식분할 내용,구분,분할전,분할후
1,1주당 가액 (원),500,100,NaN
2,발행주식총수,보통주식(주),4553965,22769825
3,종류주식(주),-,-,NaN
4,2. 주식분할 일정,주주총회예정일,2018-12-27,NaN
5,구주권제출기간,시작일,2018-12-28,NaN
6,종료일,2019-01-29,NaN,NaN
7,매매거래정지예정기간,2019-01-28 ~ 2019-02-14,NaN,NaN
8,명의개서정지기간,2019-01-30 ~ 2019-02-13,NaN,NaN
9,신주권교부예정일,2019-02-14,NaN,NaN


In [108]:
far_before = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][1].values[0])
far_before

500

In [109]:
far_after = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][2].values[0])
far_after

100

In [110]:
approve_date = contents_df[-1][contents_df[-1][0].str.contains('일정')][2].values[0]
approve_date

'2018-12-27'

In [111]:
announce_date = contents_df[-1][contents_df[-1][0].str.contains('이사회')][1].values[0]
announce_date

'2018-11-13'

#### 거래정지 첫날(close_date)

* 이 경우 매매거래 정지기간은 표기 양식이 공시에 따라 너무 다름


* 구주권 제출 기간 -종료일 을 선택할 경우 마지막 거래일을 추출할 수 있음


In [112]:
close_date = contents_df[-1][contents_df[-1][0].str.contains('종료일')][1].values[0]
close_date

'2019-01-29'

In [114]:
open_date = contents_df[-1][contents_df[-1][0].str.contains('상장')][1].values[0]
open_date

'2019-02-15'

### 해당 공시 정보 출력해보기

In [115]:
print(company, code, announce_date,far_before, far_after, approve_date,close_date,open_date)

리켐 131100 2018-11-13 500 100 2018-12-27 2019-01-29 2019-02-15


-----

# 2016년 이후 전체 액면분할 공시 출력해보기

### 전체 진행 순서

1) 공시 리스트 불러오기


2) 리스트에서 일부 항목 추출 


3) 각 공시의 서브페이지 중 컨텐츠 페이지 불러오기


4) 컨텐츠 페이지에서 주요 항목 추출


5) 추출한 내용을 dictionary 형식으로 저장



----

### 1) 공시 리스트 불러오기

#### 금일 날짜 설정

In [8]:
dt = datetime.utcnow() + timedelta(hours = 9)    # utc 시간에서 9시간 더함
today = dt.strftime('%Y%m%d')                  # str 형태로 변환
today

'20200118'

#### 리스트 url 찾기

In [9]:
url = 'http://dart.fss.or.kr/dsab002/search.ax'

#### data parameter 구하기

In [10]:
data = {
    'currentPage': '1',            # 페이지 1로 충분함
    'maxResults': '100',           # 15에서 100으로 변경
    'maxLinks': '10',
    'sort': 'date',
    'series': 'desc',
    'textCrpCik': '',
    'textCrpNm': '',
    'textPresenterNm': '',
    'startDate': '20160101',
    'endDate': today,         # 오늘 날짜로 변경
    'finalReport': 'recent',
    'typesOfBusiness': 'all',
    'corporationType': 'all',
    'closingAccountsMonth': 'all',
    'reportName': '주식분할'
    }

#### 원하는 페이지 요청

In [12]:
response = requests.post(url, data=data)

#### 페이지 응답여부 확인해 보기

In [13]:
pd.read_html(response.text)[0]

,번호,공시대상회사,보고서명,제출인,접수일자,비고
0,1,메디파트너생명공학,주식분할결정,메디파트너생명공학,2020.01.09,NaN
1,2,남영비비안,주식분할결정,남영비비안,2019.12.11,NaN
2,3,바른테크놀로지,[기재정정]주식분할결정,바른테크놀로지,2019.10.22,NaN
3,4,에이치엔티,주식분할결정,에이치엔티,2019.10.21,NaN
4,5,뉴파워프라즈마,주식분할결정,뉴파워프라즈마,2019.09.24,NaN
5,6,코스모화학,[기재정정]주식분할결정,코스모화학,2019.09.19,NaN
6,7,흥국에프엔비,[기재정정]주식분할결정,흥국에프엔비,2019.09.16,NaN
7,8,삼보산업,[기재정정]주식분할결정,삼보산업,2019.05.07,NaN
8,9,대동금속,[기재정정]주식분할결정,대동금속,2019.04.24,NaN
9,10,풀무원,[기재정정]주식분할결정,풀무원,2019.04.02,NaN


#### 페이지 파싱(BeautifulSoup 활용)

In [14]:
soup = bs(response.text, 'html.parser')

#### tr_list 설정

In [15]:
tr_list = soup.findAll('tr')

#### text 에서 불필요한 문자 remove하는 함수 정의

In [16]:
def remove_text(item) :
    return item.replace('\n', "").replace('\t', "").replace(',' ,'').replace('\xa0', '').replace('\r', '')

공시 item에서 띄어쓰기를 없애주는 함수 정의

* 공시 item이 띄어쓰기 통일이 안되어 있음

예시) "1주당 가액 (원)", "1주당 가액(원)"

In [17]:
def del_space(item) :
    return item.replace(" ", "")

#### 각 tr별로 주요 컨텐츠 추출하는 for 문 작성

In [20]:
result_list = []     # 결과를 담을 빈 리스트 생성

for tr in progressbar(tr_list[1:]) :         # tr리스트의 1번째는 컬럼명임. 그래서 두번째부터 진행
    
    td_list = tr.findAll('td')  # 각 tr item 내에서 td 리스트를 생성
    except_list = ['메디플란트', '바이오빌', '포스링크']   # 분할 철회 등으로 빼야하는 리스트 생성
    
    
    
    company = remove_text(td_list[1].text)
    code_res = requests.get('http://dart.fss.or.kr' + td_list[1].a['href'])
    code = "A" + pd.read_html(code_res.text)[0].iloc[3,1]
    title = remove_text(td_list[2].text)
    market = td_list[1].img['title']
    
    
    if ('첨부정정' in title) or ('기타법인' in market) or (company in except_list) :
        pass 
    
    else : 
        sub_url = 'http://dart.fss.or.kr' + td_list[2].a['href']


        # 하위 페이지 요청하기
        sub_response = requests.get(sub_url)
        sub_text = sub_response.text

        # url 마지막에 있는 문서번호 구하기
        check_num = str(sub_url[-14:])


        # dcmNo 구하기 
        dcmNo = re.findall(r"\b\d{7}\b",sub_text.split(check_num)[1])[0]

        # contents가 포함된 html 문서 불러오기
        contents_url = "http://dart.fss.or.kr/report/viewer.do"
        param_data = {
        'rcpNo': check_num,    # check_num 사용(앞에서 추출한 고유번호)
        'dcmNo': dcmNo,        # dcmNo 사용(앞에서 추출한 고유번호)
        'eleId': '0',
        'offset': '0',
        'length': '0',
        'dtd': 'HTML'
        }

        contents_request = requests.get(contents_url, params = param_data)
        contents_df = pd.read_html(contents_request.text)
        
        # 각 항목 추출하기

        far_before = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][1].values[0])
        far_after = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][2].values[0])
        announce_date = contents_df[-1][contents_df[-1][0].str.contains('이사회')][1].values[0]
        approve_date = contents_df[-1][contents_df[-1][0].str.contains('일정')][2].values[0]
        close_date = contents_df[-1][contents_df[-1][0].str.contains('종료일')][1].values[0]
        open_date = contents_df[-1][contents_df[-1][0].str.contains('상장')][1].values[0]
        
        result_list.append({
            '코드' : code,
            '회사명' : company, 
            '발표일' : announce_date,
            '액면가(전)' : far_before,
            '액면가(후)' : far_after, 
            '주주총회일' : approve_date,
            '거래정지일' : close_date,
            '거래재개일' : open_date
        })

        time.sleep(random.randint(50,100)/10)

100% (100 of 100) |######################| Elapsed Time: 0:13:09 Time:  0:13:09


In [21]:
result_list

[{'코드': 'A015540',
  '회사명': '메디파트너생명공학',
  '발표일': '2020-01-09',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2020-01-31',
  '거래정지일': '-',
  '거래재개일': '2020-02-21'},
 {'코드': 'A002070',
  '회사명': '남영비비안',
  '발표일': '2019-12-11',
  '액면가(전)': 1000,
  '액면가(후)': 100,
  '주주총회일': '2019-12-27',
  '거래정지일': '-',
  '거래재개일': '2020-02-06'},
 {'코드': 'A029480',
  '회사명': '바른테크놀로지',
  '발표일': '2019-06-17',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2019-11-08',
  '거래정지일': '-',
  '거래재개일': '2019-12-24'},
 {'코드': 'A176440',
  '회사명': '에이치엔티',
  '발표일': '2019-10-21',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2019-11-05',
  '거래정지일': '-',
  '거래재개일': '2019-11-21'},
 {'코드': 'A144960',
  '회사명': '뉴파워프라즈마',
  '발표일': '2019-09-24',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2019-11-08',
  '거래정지일': '-',
  '거래재개일': '2019-12-10'},
 {'코드': 'A005420',
  '회사명': '코스모화학',
  '발표일': '2019-08-23',
  '액면가(전)': 5000,
  '액면가(후)': 1000,
  '주주총회일': '2019-10-04',
  '거래정지일': '-',
  '거래재개일': '2019-11-01'},
 {'코드': 'A189980',
  '회

In [22]:
work_df = pd.DataFrame(result_list)
work_df

,거래재개일,거래정지일,발표일,액면가(전),액면가(후),주주총회일,코드,회사명
0,2020-02-21,-,2020-01-09,500,100,2020-01-31,A015540,메디파트너생명공학
1,2020-02-06,-,2019-12-11,1000,100,2019-12-27,A002070,남영비비안
2,2019-12-24,-,2019-06-17,500,100,2019-11-08,A029480,바른테크놀로지
3,2019-11-21,-,2019-10-21,500,100,2019-11-05,A176440,에이치엔티
4,2019-12-10,-,2019-09-24,500,100,2019-11-08,A144960,뉴파워프라즈마
5,2019-11-01,-,2019-08-23,5000,1000,2019-10-04,A005420,코스모화학
6,2019-10-18,-,2019-09-16,500,100,2019-09-16,A189980,흥국에프엔비
7,2019-05-20,2019-04-29,2019-02-27,500,100,2019-03-28,A009620,삼보산업
8,2019-05-08,2019-05-02,2019-03-13,5000,1000,2019-03-29,A020400,대동금속
9,2019-05-07,2019-05-02,2019-02-27,5000,500,2019-03-29,A017810,풀무원


In [23]:
work_df.columns

Index(['거래재개일', '거래정지일', '발표일', '액면가(전)', '액면가(후)', '주주총회일', '코드', '회사명'], dtype='object')

In [24]:
col = ['코드','회사명','액면가(전)','액면가(후)','발표일','주주총회일','거래정지일','거래재개일']

In [25]:
work_df = work_df[col]

In [26]:
work_df = work_df.set_index('코드')

In [27]:
work_df

,회사명,액면가(전),액면가(후),발표일,주주총회일,거래정지일,거래재개일
코드,,,,,,,
A015540,메디파트너생명공학,500,100,2020-01-09,2020-01-31,-,2020-02-21
A002070,남영비비안,1000,100,2019-12-11,2019-12-27,-,2020-02-06
A029480,바른테크놀로지,500,100,2019-06-17,2019-11-08,-,2019-12-24
A176440,에이치엔티,500,100,2019-10-21,2019-11-05,-,2019-11-21
A144960,뉴파워프라즈마,500,100,2019-09-24,2019-11-08,-,2019-12-10
A005420,코스모화학,5000,1000,2019-08-23,2019-10-04,-,2019-11-01
A189980,흥국에프엔비,500,100,2019-09-16,2019-09-16,-,2019-10-18
A009620,삼보산업,500,100,2019-02-27,2019-03-28,2019-04-29,2019-05-20
A020400,대동금속,5000,1000,2019-03-13,2019-03-29,2019-05-02,2019-05-08


### Nan 값 추출하기

In [ ]:
work_df.isnull().sum()


### 특정 값 포함했는지 확인하기

In [ ]:
work_df.isin(['-']).sum()

### 특정값 포함한 것 확인하기

In [ ]:
work_df[work_df['주주총회일'].isin(['-'])]

### 수동으로 입력하기

In [ ]:
work_df.loc[work_df['회사명']=='대양제지', '주주총회일'] = '2017-03-25'

work_df.loc[work_df['회사명']=='코리아에스이', '주주총회일'] = '2016-03-24'

In [ ]:
work_df[work_df['회사명'].isin(['대양제지', '코리아에스이'])]

### 엑셀로 보내기

In [ ]:
#work_df.to_excel('액면분할.xlsx', encoding = 'euckr')

### 엑셀에서 읽어오기

In [ ]:
work = pd.read_excel("액면분할.xlsx", encoding = 'euckr', index_col = '코드')

In [ ]:
work

In [ ]:
work[['주주총회일', '거래정지일', '거래재개일']] = work[['주주총회일', '거래정지일', '거래재개일']].apply(pd.to_datetime)

In [ ]:
work.info()